# apps
> Framework-specific Dockerfile generators for common web app patterns

In [ ]:
#| default_exp apps

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import listify
from dockr.core import Dockerfile

## Python apps

`python_app()` is the foundation — a single-stage Python Dockerfile with optional uv support.
All higher-level helpers (`fasthtml_app`) delegate to it.

In [ ]:
#| export
def python_app(port=8000, cmd=None, image='python:3.12-slim', workdir='/app',
               pkgs=None, volumes=None, uv=True, healthcheck=None):
    'Single-stage Python app Dockerfile. uv=True (default) uses uv for fast installs.'
    df = Dockerfile().from_(image).workdir(workdir)
    if pkgs:
        df = df.apt_install(*listify(pkgs), y=True).run('rm -rf /var/lib/apt/lists/*')
    if uv:
        df = (df.copy('/uv', '/usr/local/bin/uv', from_='ghcr.io/astral-sh/uv:latest')
                .copy('pyproject.toml', '.').copy('uv.lock', '.')
                .run_mount('uv sync --frozen --no-dev', target='/root/.cache/uv'))
    else:
        df = df.copy('requirements.txt', '.').run('pip install --no-cache-dir -r requirements.txt')
    df = df.copy('.', '.')
    for v in listify(volumes or []):
        df = df.run(f'mkdir -p {v}')
    if healthcheck:
        df = df.healthcheck(f'curl -f http://localhost:{port}{healthcheck}', i='30s', t='5s', r='3')
    df = df.expose(port)
    _cmd = cmd or ['python', 'main.py']
    return df.cmd(_cmd if isinstance(_cmd, list) else _cmd.split())

In [ ]:
# Defaults: uv, port 8000
df = python_app()
s = str(df)
assert 'FROM python:3.12-slim' in s
assert 'COPY --from=ghcr.io/astral-sh/uv:latest /uv /usr/local/bin/uv' in s
assert 'RUN --mount=type=cache,target=/root/.cache/uv uv sync --frozen --no-dev' in s
assert 'COPY . .' in s
assert 'EXPOSE 8000' in s
assert 'CMD ["python", "main.py"]' in s
assert 'VOLUME' not in s
print('python_app() defaults OK'); print(df)

In [ ]:
# With pip, apt packages, volumes, healthcheck
df = python_app(port=5001, uv=False,
    pkgs=['libpq-dev', 'curl'],
    volumes=['/app/data', '/app/uploads'],
    healthcheck='/health')
s = str(df)
assert 'apt-get install -y libpq-dev curl' in s
assert 'requirements.txt' in s
assert 'mkdir -p /app/data' in s
assert 'mkdir -p /app/uploads' in s
assert 'HEALTHCHECK' in s
assert 'VOLUME' not in s
print('python_app() pip+pkgs+volumes+healthcheck OK')

## FastHTML / FastAPI

`fasthtml_app()` is a thin wrapper around `python_app()` with sensible FastHTML defaults: port 5001, uv, `python main.py`.

In [ ]:
#| export
def fasthtml_app(port=5001, cmd=None, image='python:3.12-slim', pkgs=None,
                 volumes=None, healthcheck=None):
    'FastHTML/FastAPI single-stage Dockerfile with uv'
    return python_app(port=port, cmd=cmd, image=image, pkgs=pkgs,
                      volumes=volumes, uv=True, healthcheck=healthcheck)

In [ ]:
df = fasthtml_app(port=5001)
s = str(df)
assert 'EXPOSE 5001' in s
assert 'ghcr.io/astral-sh/uv:latest' in s
assert 'CMD ["python", "main.py"]' in s
print('fasthtml_app() OK'); print(df)

In [ ]:
# With extra apt packages and mounted data volume
df = fasthtml_app(
    port=5001,
    pkgs=['ca-certificates', 'rclone'],
    volumes=['/app/data'],
    healthcheck='/health')
s = str(df)
assert 'rclone' in s
assert 'mkdir -p /app/data' in s
assert 'HEALTHCHECK' in s
print('fasthtml_app() with pkgs+volumes+healthcheck OK')

## FastAPI + React (two-stage)

`fastapi_react()` builds a two-stage image: Node.js frontend build → Python backend.
The compiled frontend assets are copied into `/app/static` for the Python server to serve.

In [ ]:
#| export
def fastapi_react(port=8000, node_version='20', frontend_dir='frontend', build_dir='dist',
                  image='python:3.12-slim', pkgs=None, uv=True, healthcheck='/health'):
    'Two-stage Dockerfile: Node.js frontend build + Python/FastAPI backend'
    df = (Dockerfile()
        .from_(f'node:{node_version}-slim', as_='frontend')
        .workdir('/build')
        .copy(f'{frontend_dir}/package*.json', '.')
        .run('npm ci')
        .copy(frontend_dir, '.')
        .run('npm run build')
        .from_(image).workdir('/app'))
    if pkgs:
        df = df.apt_install(*listify(pkgs), y=True).run('rm -rf /var/lib/apt/lists/*')
    if uv:
        df = (df.copy('/uv', '/usr/local/bin/uv', from_='ghcr.io/astral-sh/uv:latest')
                .copy('pyproject.toml', '.').copy('uv.lock', '.')
                .run_mount('uv sync --frozen --no-dev', target='/root/.cache/uv'))
    else:
        df = df.copy('requirements.txt', '.').run('pip install --no-cache-dir -r requirements.txt')
    df = (df.copy('.', '.')
            .copy(f'/build/{build_dir}', '/app/static', from_='frontend'))
    if healthcheck:
        df = df.healthcheck(f'curl -f http://localhost:{port}{healthcheck}', i='30s', t='5s', r='3')
    return df.expose(port).cmd(['uvicorn', 'main:app', '--host', '0.0.0.0', f'--port={port}'])

In [ ]:
df = fastapi_react(port=8000)
s = str(df)
assert 'FROM node:20-slim AS frontend' in s
assert 'npm ci' in s
assert 'npm run build' in s
assert 'FROM python:3.12-slim' in s
assert 'COPY --from=frontend /build/dist /app/static' in s
assert 'uvicorn' in s
print('fastapi_react() OK'); print(df)

## Go

`go_app()` compiles a Go binary with `go build` and copies it into a minimal `distroless/static` image.
Module downloads are cached with `--mount=type=cache`.

In [ ]:
#| export
def go_app(port=8080, go_version='1.22', binary='app', runtime='gcr.io/distroless/static',
           cmd=None, cgo=False):
    'Two-stage Go Dockerfile: go compiler + go mod cache → distroless runtime'
    df = (Dockerfile()
        .from_(f'golang:{go_version}-alpine', as_='builder')
        .workdir('/src')
        .copy('go.mod', '.').copy('go.sum', '.')
        .run_mount('go mod download', target='/go/pkg/mod')
        .copy('.', '.')
        .env('CGO_ENABLED', '0' if not cgo else '1')
        .run('go build -ldflags="-s -w" -o /app .')
        .from_(runtime)
        .copy('/app', '/app', from_='builder')
        .expose(port))
    return df.cmd(cmd or ['/app'])

In [ ]:
df = go_app(port=8080)
s = str(df)
assert 'FROM golang:1.22-alpine AS builder' in s
assert 'RUN --mount=type=cache,target=/go/pkg/mod go mod download' in s
assert 'ENV CGO_ENABLED=0' in s
assert 'go build -ldflags' in s
assert 'FROM gcr.io/distroless/static' in s
assert 'COPY --from=builder /app /app' in s
assert 'CMD ["/app"]' in s
print('go_app() OK'); print(df)

## Rust

`rust_app()` compiles a Rust binary in release mode and copies it into a minimal image.
The cargo registry is cached with `--mount=type=cache` to avoid re-downloading crates.

In [ ]:
#| export
def rust_app(port=8080, rust_version='1', binary='app', runtime='gcr.io/distroless/static',
             features=None, release=True):
    'Two-stage Rust Dockerfile: cargo build → distroless runtime'
    build_cmd = 'cargo build --release' + (f' --features {features}' if features else '')
    df = (Dockerfile()
        .from_(f'rust:{rust_version}-slim-bookworm', as_='builder')
        .workdir('/src')
        .copy('.', '.')
        .run_mount(build_cmd, target='/usr/local/cargo/registry')
        .from_(runtime)
        .copy(f'/src/target/release/{binary}', f'/{binary}', from_='builder')
        .expose(port))
    return df.cmd([f'/{binary}'])

In [ ]:
df = rust_app(port=8080)
s = str(df)
assert 'FROM rust:1-slim-bookworm AS builder' in s
assert 'RUN --mount=type=cache,target=/usr/local/cargo/registry cargo build --release' in s
assert 'FROM gcr.io/distroless/static' in s
assert 'COPY --from=builder /src/target/release/app /app' in s
assert 'CMD ["/app"]' in s
print('rust_app() OK'); print(df)

In [ ]:
# With features
df = rust_app(binary='myapp', features='postgres,redis')
s = str(df)
assert '--features postgres,redis' in s
assert '/src/target/release/myapp' in s
print('rust_app() features OK')

## VedicReader example

How `fasthtml_app()` simplifies a real production Dockerfile.
Credentials stay out of the image — they're injected at runtime via environment variables or mounted config files.

In [ ]:
df = fasthtml_app(
    port=5001,
    image='python:3.13-slim-bookworm',
    pkgs=['ca-certificates', 'rclone', 'gettext-base', 'libsqlite3-dev'],
    volumes=['/app/data', '/app/backups'],
    healthcheck='/health')
s = str(df)
assert 'python:3.13-slim-bookworm' in s
assert 'rclone' in s
assert 'mkdir -p /app/data' in s
assert 'HEALTHCHECK' in s
print('vedicreader-style fasthtml_app() OK'); print(df)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()